# Currying in elixir

---

## Financial Team TWIL Shared

> Shared by [laserx](https://github.com/laserx)

## References

* [Function currying in Elixir](http://blog.patrikstorm.com/function-currying-in-elixir)
* [repo: Qqwy/elixir_currying](https://github.com/Qqwy/elixir_currying)

---

## Currying

Currying is a technique that translates a function of arity N into a sequence of functions that all have the arity 1. Function arity means the number of arguments a function takes.

### A naive approach

Elixir functions are first class, so your function can return them and receive them as values.

In [1]:
add = fn x ->
  fn y -> x + y end
end

#Function<6.128620087/1 in :erl_eval.expr/5>

In [2]:
increment = add.(1)

#Function<6.128620087/1 in :erl_eval.expr/5>

In [3]:
increment.(4)

5

This works, but its obvious that we want something more general, and reusable. In fact the above could be considered a plain function that is partially applied.

### We can do better

In [4]:
defmodule Curry do
  def curry(fun) do
    {_, arity} = :erlang.fun_info(fun, :arity)
    curry(fun, arity, [])
  end

  def curry(fun, 0, arguments) do
    apply(fun, Enum.reverse arguments)
  end

  def curry(fun, arity, arguments) do
    fn arg -> curry(fun, arity - 1, [arg | arguments]) end
  end
end

{:module, Curry, <<70, 79, 82, 49, 0, 0, 6, 88, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 184, 0, 0, 0, 20, 12, 69, 108, 105, 120, 105, 114, 46, 67, 117, 114, 114, 121, 8, 95, 95, 105, 110, 102, 111, 95, 95, 7, ...>>, {:curry, 3}}

In [5]:
import Curry

curried = curry(fn a, b, c, d -> a * b + div(c, d) end)

c = curried.(5).(5).(10)

#Function<0.73188802/1 in Curry.curry/3>

In [6]:
c.(1)

35

In [7]:
import Curry

enum_map = curry(&Enum.map/2)

#Function<0.73188802/1 in Curry.curry/3>

In [8]:
enum_map.([1, 2, 3, 4, 5]).(fn x -> x * x end)

[1, 4, 9, 16, 25]